In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers

In [2]:
data_train_prepared = pd.read_csv("../processed_data/data_train_prepared.csv")
y_train = pd.read_csv("../processed_data/y_train.csv")
data_test_prepared = pd.read_csv("../processed_data/data_test_prepared.csv")
y_test = pd.read_csv("../processed_data/y_test.csv")

In [3]:
num_samples_train, num_features_train = data_train_prepared.shape
num_samples_test, num_features_test = data_test_prepared.shape
timesteps = 300

num_sequences_train = num_samples_train // timesteps
num_sequences_test = num_samples_test // timesteps

# Reshape data to 3D: (num_sequences, timesteps, num_features) to use LSTM layer
data_train_3D = data_train_prepared.values[:num_sequences_train * timesteps].reshape((num_sequences_train, timesteps, num_features_train))
data_test_3D = data_test_prepared.values[:num_sequences_test * timesteps].reshape((num_sequences_test, timesteps, num_features_test))

In [4]:
### Using LSTM ### 

# Define the model
model_lstm = keras.Sequential()

# Add a LSTM layer with 128 internal units.
model_lstm.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True), input_shape=(timesteps, num_features_train)))
model_lstm.add(layers.Dropout(0.5))

# Add another LSTM layer
model_lstm.add(layers.Bidirectional(layers.LSTM(64)))
model_lstm.add(layers.Dropout(0.5))

# Add a Dense layer with 5 units.
model_lstm.add(layers.Dense(y_train.shape[1]))

# Display the model summary
print(model_lstm.summary())

# Training the model

batch_size = 64

model_lstm.compile(
    loss=keras.losses.MeanAbsolutePercentageError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.3),
    metrics=["mae"],
)


model_lstm.fit(
    data_train_3D,
    y_train[:num_sequences_train],
    validation_data=(data_test_3D, y_test[:num_sequences_test]),
    batch_size=batch_size,
    epochs=20,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 300, 256)         187392    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 300, 256)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 352,389
Trainable params: 352,389
Non-trai

In [5]:
### Using GRU ###

# Define the model
model_gru = keras.Sequential()

# Add a GRU layer
model_gru.add(layers.GRU(128, input_shape=(timesteps, num_features_train)))
model_gru.add(layers.Dropout(0.5))

# Add a Dense layer
model_gru.add(layers.Dense(y_train.shape[1]))

# Display the model summary
print(model_gru.summary())

# Compile the model
batch_size = 64
model_gru.compile(
    loss=keras.losses.MeanAbsolutePercentageError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    metrics=["mae"],
)

# Fit the model
model_gru.fit(
    data_train_3D,
    y_train[:num_sequences_train],
    validation_data=(data_test_3D, y_test[:num_sequences_test]),
    batch_size=batch_size,
    epochs=20
)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 128)               70656     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 71,301
Trainable params: 71,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
2/2 [==============================] - 2s 601ms/step - loss: 475.3690 - mae: 21.8109 - val_loss: 392.8328 - val_mae: 21.7347
Epoch 2/20
2/2 [==============================] - 0s 198ms/step - loss: 448.6837 - mae: 23.6634 - val_loss: 158.2043 - val_mae: 19.5522
Epoch 3/20
2/2 [=====================